In [ ]:
import pandas as pd
import re
from ast import literal_eval

df=# load your data
df.unigrams = df.unigrams.apply(literal_eval)
df.bigrams = df.bigrams.apply(literal_eval)

In [ ]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel 
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/distilrubert-tiny-cased-conversational-v1", model_max_length=512)
model = AutoModel.from_pretrained("DeepPavlov/distilrubert-tiny-cased-conversational-v1", num_labels=2)

In [47]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [ ]:
import emoji
import re

def remove_punctuation(text):
    return re.sub(r"[,.;@#?!&$]+\ *", " ", text)

def emoji_to_text(text):
    emojied_text=emoji.demojize(text)
    return emojied_text.replace(":", " ")

#---

def pre_process_text(text):
    
    text=emoji_to_text(text)
    text = remove_punctuation(text.lower())
    text_=re.sub("\[.*?\]", "", text)
    return " ".join(text_.split())

In [49]:
df = df.fillna('')
data_cleaned=list((df['text']).apply(pre_process_text))

In [ ]:
import torch

#%%time
from tqdm.notebook import tqdm
emb_list = list()
it = 0
for s in tqdm(data_cleaned):
    emb_list.append(embed_bert_cls(s, model, tokenizer))

In [ ]:
# SAVE embeddings
file_name=#file to save emb
np.save(file_name, emb_list)

In [ ]:
import joblib

# Load model to predict political posts:
# (The model was trained on a sample that was first labeled based on a list of political words (see file '../dictionaries/political-key-words.txt'), 
# then the data was manually checked for false positives and false negatives. 
# Then, based on distilrubert-tiny-cased-conversational-v1 embeddings, SVM model was built.)

file_name='../models/SVC_for_political_classification.pkl'
loaded_model = joblib.load(file_name)

In [ ]:
# Predict 
y_predict = (loaded_model.predict_proba(emb_list)[:,1] >= 0.4).astype(int)
df['If_political']=y_predict

In [ ]:
df['If_political'].value_counts()

In [ ]:
#save df